# Introduction
In this project, I will create an AI asisstant for PDF summarization. Most PDFs are too long to read, and i don't have a time. Thanks to this project, I will be able to summary before going through the entire PDF and decide whether it's worth reading fully or not.

In [1]:
# Library
import os
import requests
from dotenv import load_dotenv
from IPython.display import Markdown, display
from openai import OpenAI
import fitz

In [2]:
# checking api key
load_dotenv(override = True)
api_key = os.getenv('OPENAI_API_KEY')

if api_key and api_key.startswith('sk-proj-') and len(api_key)>10:
    print("API key looks good so far")
else:
    print("There might be a problem with your API key? Please visit the troubleshooting notebook!")

API key looks good so far


In [3]:
openai = OpenAI()

In [5]:
doc = fitz.open("D:\\work\\Project\\LLMEngineer\\llm_engineering\\week1\\Article1.pdf")

In [6]:
# I used the fitz library for the read article.
text = ""
for page in doc:
    text += page.get_text()
print(text)

SYSTEMATIC REVIEW
Open Access
© The Author(s) 2025. Open Access  This article is licensed under a Creative Commons Attribution-NonCommercial-NoDerivatives 4.0 
International License, which permits any non-commercial use, sharing, distribution and reproduction in any medium or format, as long as you 
give appropriate credit to the original author(s) and the source, provide a link to the Creative Commons licence, and indicate if you modified the 
licensed material. You do not have permission under this licence to share adapted material derived from this article or parts of it. The images or 
other third party material in this article are included in the article’s Creative Commons licence, unless indicated otherwise in a credit line to the 
material. If material is not included in the article’s Creative Commons licence and your intended use is not permitted by statutory regulation or 
exceeds the permitted use, you will need to obtain permission directly from the copyright holder. To view

In [16]:
# I prepared the system prompt for AI asisstant
system_prompt = "You are an AI assistant specialized in reading and summarizing long texts, such as academic articles, research papers, and reports. \
Your task is to read the given document carefully and generate a clear, concise, and informative summary. Focus on capturing the main points, key findings,\
and essential information. Do not copy text directly; instead, paraphrase in a natural and readable way.\
The summary should help the reader quickly understand the purpose and content of the original document and decide whether it is worth reading in full."

# I updated the system prompt for AI asistant give Turkish language.

system_prompt_v2 = """You are an AI assistant specialized in reading and summarizing long texts such as academic articles, research papers, and reports. \
Your task is to read the given document carefully and generate a clear, concise, and informative summary. Focus on capturing the main points, key arguments, \
and important findings. Do not copy text directly; instead, paraphrase in a natural and readable way.

You can produce summaries in English or Turkish depending on the user’s prompt. If the user requests it in Turkish, write the summary in Turkish. \
Otherwise, default to English.

The summary should help the user quickly understand the content and purpose of the original document and decide whether to read it fully."""

In [17]:
system_prompt_v2

'You are an AI assistant specialized in reading and summarizing long texts such as academic articles, research papers, and reports. Your task is to read the given document carefully and generate a clear, concise, and informative summary. Focus on capturing the main points, key arguments, and important findings. Do not copy text directly; instead, paraphrase in a natural and readable way.\n\nYou can produce summaries in English or Turkish depending on the user’s prompt. If the user requests it in Turkish, write the summary in Turkish. Otherwise, default to English.\n\nThe summary should help the user quickly understand the content and purpose of the original document and decide whether to read it fully.'

In [21]:
# I prepared the user prompt for AI asisstant
def user_prompt_for(article_text, language):
    user_prompt = """Please read the following document and provide a well-structured summary. \
        Focus on the main ideas, important arguments, and key findings. \
        Your summary should be clear and concise, and written in natural language. \
        Avoid copying long passages from the text. \
        The goal is to help me quickly understand what the document is about. """
    user_prompt += f"Please prepare the document with {language} language\n\n"
    user_prompt += f"Document:\n{article_text}"
    return user_prompt

In [22]:
print(user_prompt_for(text, language = 'Turkish'))

Please read the following document and provide a well-structured summary.         Focus on the main ideas, important arguments, and key findings.         Your summary should be clear and concise, and written in natural language.         Avoid copying long passages from the text.         The goal is to help me quickly understand what the document is about. Please prepare the document with Turkish language

Document:
SYSTEMATIC REVIEW
Open Access
© The Author(s) 2025. Open Access  This article is licensed under a Creative Commons Attribution-NonCommercial-NoDerivatives 4.0 
International License, which permits any non-commercial use, sharing, distribution and reproduction in any medium or format, as long as you 
give appropriate credit to the original author(s) and the source, provide a link to the Creative Commons licence, and indicate if you modified the 
licensed material. You do not have permission under this licence to share adapted material derived from this article or parts of it.

In [27]:
def massages_for(article_text, language):
    return [
        {'role': 'system', 'content': system_prompt},
        {'role': 'user', 'content': user_prompt_for(article_text, language = language)}
    ]

In [28]:
massages_for(text, 'Turkish')

[{'role': 'system',
  'content': 'You are an AI assistant specialized in reading and summarizing long texts, such as academic articles, research papers, and reports. Your task is to read the given document carefully and generate a clear, concise, and informative summary. Focus on capturing the main points, key findings,and essential information. Do not copy text directly; instead, paraphrase in a natural and readable way.The summary should help the reader quickly understand the purpose and content of the original document and decide whether it is worth reading in full.'},
 {'role': 'user',
  'content': 'Please read the following document and provide a well-structured summary.         Focus on the main ideas, important arguments, and key findings.         Your summary should be clear and concise, and written in natural language.         Avoid copying long passages from the text.         The goal is to help me quickly understand what the document is about. Please prepare the document wit

In [33]:
def summarize(text, language):
    response = openai.chat.completions.create(
        model = 'gpt-4o-mini',
        messages = massages_for(text, language)
    )
    return response.choices[0].message.content

In [34]:
summarize(text, 'Turkish')

"**Özet**\n\n**Arka Plan:** Büyük Diller Modelleri (LLM'ler), dönüştürücü mimariler üzerine inşa edilen gelişmiş yapay zeka araçlarıdır ve klinik tıpta karar verme desteği, tanı ve tıbbi eğitim gibi alanlarda önemli potansiyellere sahiptir. Ancak, klinik iş akışlarına entegrasyonları için güvenilirlik, güvenlik ve etik uyum açısından titiz bir değerlendirme gereklidir.\n\n**Amaç:** Bu sistematik inceleme, klinik tıpta LLM'ler hakkında yapılan değerlendirme parametrelerini ve metodolojileri inceleyerek yeteneklerini, sınırlamalarını ve uygulama trendlerini vurgular.\n\n**Yöntem:** PubMed, Scopus, Web of Science, IEEE Xplore ve arXiv veri tabanlarında kapsamlı bir literatür taraması gerçekleştirildi. Önceden belirlenen dahil etme ve hariç tutma kriterlerine göre, tıbbi bağlamlarda LLM performansını değerlendiren özgün araştırmalar belirlendi. \n\n**Sonuçlar:** Tarama sonucunda 761 çalışma seçilmiş olup, genel alan LLM'leri (özellikle ChatGPT ve GPT-4) %93.55 oranla, tıbbi alan LLM'leri i

In [37]:
def display_summary(text, language):
    summary = summarize(text, language)
    display(Markdown(summary))

In [38]:
display_summary(text, 'Turkish')

**Büyük Dil Modellerinin Klinik Tıptaki Değerlendirmeleri Üzerine Sistematik İnceleme**

**Özeti:**
Bu sistematik inceleme çalışması, Klinik Tıp'ta büyük dil modellerinin (LLM) değerlendirilmesine yönelik parametreler ve metodolojileri incelemektedir. Bu modellerin (örneğin ChatGPT, GPT-4) klinik karar destek sistemlerinde, tanılarda ve tıbbi eğitimdeki potansiyeli vurgulanırken, güvenilirlik, güvenlik ve etik uygunluk konularının titiz değerlendirmelere tabi tutulması gerektiği belirtilmektedir.

**Araştırma Amacı:**
LLM'lerin sağlık alanındaki değerlendirme yöntemlerini ve uygulama alanlarını belirlemek amacıyla yapılan bu çalışmada, 761 ilgili çalışma incelenmiştir. Araştırma, genel alan LLM'lerinin (özellikle ChatGPT ve GPT-4) değerlendirmelerde baskın olduğunu, ancak tıbbi alanda kullanılan LLM'lerin oldukça az olduğunu göstermektedir.

**Metodoloji:**
Literatür incelemesi, PubMed, Scopus, Web of Science, IEEE Xplore ve arXiv gibi veritabanları üzerinden gerçekleştirilmiştir. Seçilen çalışmalar, önceden tanımlanmış kriterlere göre süzgeçten geçirilmiş ve yalnızca orijinal araştırmalar dahil edilmiştir.

**Ana Bulgular:**
- Eğlence veya genel alan LLM'leri üzerine yapılan çalışmalar 761 pozitif sonuçla belirlenmiştir.
- Doğruluk, en yaygın değerlendirme parametresi olmuştur.
- LLM'ler arasında en çok değerlendirilen yapı, genel alan LLM'leri ile özellikle decoders-only modeller olmuştur.
- Klinik uygulamalarda bazı önemli uzmanlık alanlarının yeterince temsil edilmediği görülmektedir.
  
**Sonuçlar:**
LLM'lerin klinik tıptaki rolleri hızla artmaktadır, ancak eğitim ve güvenlik konularındaki zorluklar aşılmalıdır. Gelecek çalışmaların standartlaştırılmış değerlendirme çerçevelerine odaklanması ve kritik uzmanlık alanlarına yönelmesi gerektiği belirtilmektedir. 

**Anahtar Kelimeler:**
Büyük dil modelleri, LLM değerlendirmesi, Klinik tıp, Yapay zeka, Tıbbi eğitim.

In [39]:
display_summary(text, 'English')

The document is a systematic review that investigates the evaluation of Large Language Models (LLMs) in clinical medicine. It highlights both the burgeoning interest in LLMs for enhancing decision support, diagnostics, and medical education, as well as the need for rigorous assessments to ensure their safety, reliability, and ethical use.

### Purpose
The primary objective of this review is to examine the evaluation parameters and methodologies used for assessing LLMs in healthcare settings. It aims to summarize the capabilities, limitations, and trends in the application of these models in clinical contexts.

### Methods
The review encompasses literature from various databases (PubMed, Scopus, Web of Science, IEEE Xplore, and arXiv) and includes both peer-reviewed and preprint studies. The researchers applied strict inclusion and exclusion criteria to identify 761 original research articles pertinent to the evaluations of LLMs in medical contexts.

### Key Findings
1. **Growth in Research**: There is a notable increase in studies focused on LLMs, rising dramatically from a mere 1 publication in 2019 to 557 in 2024, signifying growing interest and utility in healthcare.

2. **Model Types**: The majority of the evaluated models (93.55%) were general-domain LLMs such as ChatGPT and GPT-4, while only 6.45% pertained to medical-domain LLMs. Among medical models, decoder-only architectures were the most common.

3. **Evaluation Metrics**: The most frequently assessed parameter was accuracy, appearing in 21.78% of the evaluations. Other parameters like consistency, performance, and clarity were also noted but were less emphasized.

4. **Specialty Representation**: Surgical specialties were the most frequently evaluated (28.2% of studies), particularly ophthalmology and orthopedics. Critical specialties such as cardiology and emergency medicine were notably underrepresented.

5. **Target Audience**: The targeted evaluations primarily included doctors (31.4%), followed by patients (26.6%). The studies showed a diverse range of applications and audiences, including educational uses and examination purposes in medicine.

### Conclusions
The review emphasizes the transformative potential of LLMs in healthcare but also points out significant challenges, such as variability in evaluation methods, the need for standardized frameworks, and ethical concerns surrounding bias and data privacy. Addressing these issues is crucial for the responsible integration of LLMs into clinical practice. The authors recommend future research to focus on model interpretability and developing guidelines that align LLM assessments with specific clinical needs.

Overall, while LLMs show promise in revolutionizing healthcare, careful and ethical evaluation frameworks must accompany their advancement to maximize benefits in medical practice.

# Conclusion
I finished this project. I used two languages in this project. The results may differ from each other because we send a summary request to the AI model for each summary preparation. <br>
I am improving myself on LLM models and I will prepare the same project. I am new to this field. I may have made mistakes. I am sorry about that. 
I hope my project was useful for you. Thank you for taking a look at my project. I will continue to share projects. If you want to be informed in advance,  you can follow me from the links below.

[LinkedIn](https://www.linkedin.com/in/ihsancenkiz/)
[GitHub](https://github.com/ihsncnkz)
[Kaggle](https://www.kaggle.com/ihsncnkz)